In [0]:
import h5py
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from tqdm import tqdm_notebook as tqdm
import torch.utils.data as utils
from torch.autograd import Variable
from torchvision import transforms
from torch import nn, optim
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
x_train  = np.load('/gdrive/My Drive/data/Sxx.npy')
x_train0 = np.load('/gdrive/My Drive/data/head.npy')

x_test0  = x_train0[183143:]
x_test   = x_train[183143:]
x_train  = x_train[:183143]
x_train0 = x_train0[:183143]

In [0]:
x_test = np.load('/gdrive/My Drive/data/Sxx.npy')[200000:220000]
y_test = pd.read_csv("/gdrive/My Drive/data/y_train.csv").values[200000:220000, 1].squeeze()

In [0]:

x_test  = torch.tensor(x_test).float()
test_mean  = x_test.mean(dim = (1, 2), keepdim=True)
test_std   = x_test.std(dim = (1, 2) ,keepdim=True)
x_test  = (x_test - test_mean) / test_std
x_test  = x_test.view([-1, 1, 65, 11])

In [0]:
y_train = pd.read_csv("/gdrive/My Drive/data/y_train.csv").values[:183143, 1].squeeze()
y_test = pd.read_csv("/gdrive/My Drive/data/y_train.csv").values[183143:, 1].squeeze()

In [0]:
y_train = torch.tensor(y_train).long().cuda()
y_test  = torch.tensor(y_test).long().cuda()

In [0]:
x_train  = torch.tensor(x_train).float().cuda()
x_train0 = torch.tensor(x_train0).float().cuda()
x_test   = torch.tensor(x_test).float().cuda()
x_test0  = torch.tensor(x_test0).float().cuda()

In [0]:
test_mean   = x_test.mean(dim = 0, keepdim=True)
test_std    = x_test.std(dim = 0,keepdim=True)
train_mean  = x_train.mean(dim = 0,keepdim=True)
train_std   = x_train.std(dim = 0,keepdim=True)
test_mean0  = x_test0.mean(dim = 0, keepdim=True)
test_std0   = x_test0.std(dim = 0,keepdim=True)
train_mean0 = x_train0.mean(dim = 0,keepdim=True)
train_std0  = x_train0.std(dim = 0,keepdim=True)

In [0]:
x_test   = (x_test - test_mean) / test_std
x_train  = (x_train - train_mean) / train_std
x_test0  = (x_test0 - test_mean0) / test_std0
x_train0 = (x_train0 - train_mean0) / train_std0

In [0]:
x_train  = x_train.view([-1, 1, 65, 11])
x_test   = x_test.view([-1, 1, 65, 11])
x_train0 = x_train0.view([-1, 11])
x_test0  = x_test0.view([-1, 11])

In [0]:
train_data = utils.TensorDataset(x_train0, x_train, y_train)
test_data  = utils.TensorDataset(x_test0, x_test, y_test)

In [0]:
train_loader = utils.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = utils.DataLoader(test_data, batch_size=64, shuffle=True)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, (6, 2))
        self.conv2 = nn.Conv2d(16, 64, (4, 2))
        self.conv3 = nn.Conv2d(64, 128, (3, 3))
        self.fc1   = nn.Linear(2560, 512)
        self.fc2   = nn.Linear(512, 11)
        self.fc3   = nn.Linear(22, 22)
        self.fc4   = nn.Linear(22, 16)
        self.fc5   = nn.Linear(16, 3)
        self.drop_layer1 = nn.Dropout(p = 0.2)
        self.drop_layer2 = nn.Dropout(p = 0.5)
        
    def forward(self, data, x):
        x = self.conv1(x)
        x = torch.cat((x[:, :, :30, :], F.max_pool2d(x[:, :, 30:, :], (3, 1), (3, 1))), 2)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2, (2, 1), padding = (0, 1))
        x = self.conv3(x)
        x = F.max_pool2d(x, (3, 2), (3, 2))
        x = x.view([-1, 2560])
        x = self.fc1(x)
        x = F.relu(x)
        x = self.drop_layer1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = torch.cat((data, x), dim = 1)
        x = self.drop_layer1(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.drop_layer2(x)
        x = self.fc4(x)
        x = F.relu(x)
        x = self.drop_layer2(x)
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

In [0]:
loss_function = nn.NLLLoss()

In [0]:
model = Net()

In [47]:
model.cuda()

Net(
  (conv1): Conv2d(1, 16, kernel_size=(6, 2), stride=(1, 1))
  (conv2): Conv2d(16, 64, kernel_size=(4, 2), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2560, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=11, bias=True)
  (fc3): Linear(in_features=22, out_features=22, bias=True)
  (fc4): Linear(in_features=22, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=3, bias=True)
  (drop_layer1): Dropout(p=0.2)
  (drop_layer2): Dropout(p=0.5)
)

In [0]:
optimizer = optim.Adam(model.parameters())

In [0]:
epochs = 50
for e in range(epochs):
    running_loss = 0
    for data, img, labels in train_loader:
        optimizer.zero_grad()
        output = model(data, img)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_loader)))

Epoch 0 - Training loss: 1.0402702856613655
Epoch 1 - Training loss: 1.03282686734766
Epoch 2 - Training loss: 1.0312119822748718
Epoch 3 - Training loss: 1.0295170493304189
Epoch 4 - Training loss: 1.029524381243208
Epoch 5 - Training loss: 1.02970760690008
Epoch 6 - Training loss: 1.029731024111342
Epoch 7 - Training loss: 1.029576537431661
Epoch 8 - Training loss: 1.0292952571566667
Epoch 9 - Training loss: 1.0287545086577754
Epoch 10 - Training loss: 1.0285797331359818
Epoch 11 - Training loss: 1.0292416413416818
Epoch 12 - Training loss: 1.0287510294292792
Epoch 13 - Training loss: 1.0290169020702087
Epoch 14 - Training loss: 1.0293236252396862
Epoch 15 - Training loss: 1.0292841537610207
Epoch 16 - Training loss: 1.028891286072741
Epoch 17 - Training loss: 1.0292404462081581
Epoch 18 - Training loss: 1.0286990354770884
Epoch 19 - Training loss: 1.0297793952174157
Epoch 20 - Training loss: 1.029500678712717
Epoch 21 - Training loss: 1.0291226397728603
Epoch 22 - Training loss: 1.0

In [0]:
torch.save(model.state_dict(), '/gdrive/My Drive/data/Second')

In [0]:
model.load_state_dict(torch.load('/gdrive/My Drive/data/Second', map_location=lambda storage, loc: storage))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
model.eval()

Net(
  (conv1): Conv2d(1, 16, kernel_size=(6, 2), stride=(1, 1))
  (conv2): Conv2d(16, 64, kernel_size=(4, 2), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2560, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
)

In [0]:
y_pred = []
for data in tqdm(x_test):
  res = torch.exp(model(data.view([1, 1, 65, 11])))
  y_pred.append(res)

In [0]:
test = []
for data in tqdm(y_pred):
  test.append(np.argmax(data.detach().numpy()))
test = np.array(test)

In [0]:
test = np.argmax(y_pred.detach().numpy(), axis = 1)

In [0]:
np.sum(test == y_test)/len(y_test)

0.48915

In [0]:
np.sum(test == y_test)/len(y_test)

0.5131034131301678

In [0]:
df = pd.DataFrame()
df['id'] = range(len(test))
df['label'] = test
df.to_csv('./data/result.csv')